In [3]:
import tensorflow as tf
import experiment_utils as eu
import sys
sys.path.append('../')
from data_processing.stl10 import get_stl10
import importlib
import config as conf

In [4]:
importlib.reload(eu)
config = conf.get_byol()
config.batch_size = 10
ds, _ = get_stl10(batch_size=config.batch_size, split='train')
ds_small = ds.take(15)
experiment = eu.Experiment(config)
experiment.train(ds_small, 'saved_model', epochs=1)


  7%|▋         | 1/15 [00:04<01:04,  4.60s/it]

Loss: -0.3784692883491516


 13%|█▎        | 2/15 [00:08<00:52,  4.06s/it]

Loss: -0.17103311419487


 20%|██        | 3/15 [00:12<00:47,  3.93s/it]

...

Loss: -1.798607587814331


100%|██████████| 15/15 [00:56<00:00,  3.80s/it]

Loss: -1.8529866933822632
Epoch 000: Loss: -1.619


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


In [3]:
y_true = [[0., 1.], [1., 1.]]
y_pred = [[1., 0.], [1., 1.]]
# Using 'auto'/'sum_over_batch_size' reduction type.
cosine_loss = tf.keras.losses.CosineSimilarity(axis=1)
cosine_loss